# Hyperparameter Optimization (HPO) of Machine Learning Models
L. Yang and A. Shami, "On Hyperparameter Optimization of Machine Learning Algorithms: Theory and Practice," Neurocomputing, 2020.

### **Sample code for regression problems**  
**Dataset used:**  
&nbsp; Boson Housing dataset from sklearn

**Machine learning algorithms used:**  
&nbsp; Random forest (RF), support vector machine (SVM), k-nearest neighbor (KNN)

**HPO algorithms used:**  
&nbsp; Grid search, random search, hyperband, Bayesian Optimization with Gaussian Processes (BO-GP), Bayesian Optimization with Tree-structured Parzen Estimator (BO-TPE), particle swarm optimization (PSO), genetic algorithm (GA).

**Performance metric:**  
&nbsp; Mean square error (MSE)

In [72]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC,SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import datasets
import scipy.stats as stats

In [5]:
import warnings
warnings.filterwarnings("ignore")

## Load Boston Housing dataset

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html  
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

In [2]:
X, y = datasets.load_boston(return_X_y=True)

In [3]:
datasets.load_boston()

{'DESCR': ".. _boston_dataset:\n\nBoston house prices dataset\n---------------------------\n\n**Data Set Characteristics:**  \n\n    :Number of Instances: 506 \n\n    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.\n\n    :Attribute Information (in order):\n        - CRIM     per capita crime rate by town\n        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.\n        - INDUS    proportion of non-retail business acres per town\n        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)\n        - NOX      nitric oxides concentration (parts per 10 million)\n        - RM       average number of rooms per dwelling\n        - AGE      proportion of owner-occupied units built prior to 1940\n        - DIS      weighted distances to five Boston employment centres\n        - RAD      index of accessibility to radial highways\n        - TAX      full-value property-tax rate p

## Baseline Machine Learning models: Regressors with Default Hyperparameters

In [31]:
#Random Forest
clf = RandomForestRegressor()
clf.fit(X,y)
scores = cross_val_score(clf, X, y, cv=3,scoring='neg_mean_squared_error')
print("MSE:"+ str(-scores.mean()))

MSE:29.596121527190764


In [32]:
#SVM
clf = SVR(gamma='scale')
clf.fit(X,y)
scores = cross_val_score(clf, X, y, cv=3,scoring='neg_mean_squared_error')
print("MSE:"+ str(-scores.mean()))

MSE:77.42951812579331


In [33]:
#KNN
clf = KNeighborsRegressor()
clf.fit(X,y)
scores = cross_val_score(clf, X, y, cv=3,scoring='neg_mean_squared_error')
print("MSE:"+ str(-scores.mean()))

MSE:81.48773186343571


## HPO Algorithm 1: Grid Search

In [34]:
from sklearn.model_selection import GridSearchCV
rf_params = {
    'n_estimators': [10, 20, 30],
    #'max_features': ['sqrt',0.5],
    'max_depth': [15,20,30,50],
    #'min_samples_leaf': [1,2,4,8],
    #"bootstrap":[True,False],
    #"criterion":['gini','entropy']
}
clf = RandomForestRegressor(random_state=0)
grid = GridSearchCV(clf, rf_params, cv=3, scoring='neg_mean_squared_error')
grid.fit(X, y)
print(grid.best_params_)
print("MSE:"+ str(-grid.best_score_))

{'max_depth': 15, 'n_estimators': 20}
MSE:29.02394449507633


In [35]:
from sklearn.model_selection import GridSearchCV
rf_params = {
    'C': [1,10, 100],
    "kernel":['poly','rbf','sigmoid'],
    "epsilon":[0.01,0.1,1]
}
clf = SVR(gamma='scale')
grid = GridSearchCV(clf, rf_params, cv=3, scoring='neg_mean_squared_error')
grid.fit(X, y)
print(grid.best_params_)
print("MSE:"+ str(-grid.best_score_))

{'kernel': 'poly', 'epsilon': 0.01, 'C': 100}
MSE:67.07483887754718


In [36]:
from sklearn.model_selection import GridSearchCV
rf_params = {
    'n_neighbors': [2, 3, 5,7,10]
}
clf = KNeighborsRegressor()
grid = GridSearchCV(clf, rf_params, cv=3, scoring='neg_mean_squared_error')
grid.fit(X, y)
print(grid.best_params_)
print("MSE:"+ str(-grid.best_score_))

{'n_neighbors': 5}
MSE:81.52933517786562


## HPO Algorithm 2: Random Search

In [37]:
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'n_estimators': sp_randint(10,100),
    "max_features":sp_randint(1,13),
    'max_depth': sp_randint(5,50),
    "min_samples_split":sp_randint(2,11),
    "min_samples_leaf":sp_randint(1,11),
    #"criterion":['gini','entropy']
}
n_iter_search=20
clf = RandomForestRegressor(random_state=0)
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='neg_mean_squared_error')
Random.fit(X, y)
print(Random.best_params_)
print("MSE:"+ str(-Random.best_score_))

{'max_depth': 49, 'min_samples_leaf': 2, 'min_samples_split': 9, 'n_estimators': 90, 'max_features': 7}
MSE:26.825166966741307


In [38]:
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'C': stats.uniform(0,50),
    "kernel":['poly','rbf','sigmoid'],
    "epsilon":stats.uniform(0,1)
}
n_iter_search=20
clf = SVR(gamma='scale')
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='neg_mean_squared_error')
Random.fit(X, y)
print(Random.best_params_)
print("MSE:"+ str(-Random.best_score_))

{'kernel': 'poly', 'epsilon': 0.49678099309788193, 'C': 27.417074148575495}
MSE:60.03157881614154


In [39]:
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'n_neighbors': range(1,20),
}
n_iter_search=10
clf = KNeighborsRegressor()
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='neg_mean_squared_error')
Random.fit(X, y)
print(Random.best_params_)
print("MSE:"+ str(-Random.best_score_))

{'n_neighbors': 13}
MSE:80.7723025469514


## HPO Algorithm 3: Hyperband

In [40]:
from hyperband import HyperbandSearchCV
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
rf_params = {
    'n_estimators': sp_randint(10,100),
    "max_features":sp_randint(1,13),
    'max_depth': sp_randint(5,50),
    "min_samples_split":sp_randint(2,11),
    "min_samples_leaf":sp_randint(1,11),
    #"criterion":['gini','entropy']
}
clf = RandomForestRegressor(random_state=0)
hyper = HyperbandSearchCV(clf, param_distributions =rf_params,cv=3,min_iter=10,max_iter=100,scoring='neg_mean_squared_error')
hyper.fit(X, y)
print(hyper.best_params_)
print("MSE:"+ str(-hyper.best_score_))

{'max_depth': 45, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 33, 'max_features': 5}
MSE:26.17217675246202


In [41]:
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
rf_params = {
    'C': stats.uniform(0,50),
    "kernel":['poly','rbf','sigmoid'],
    "epsilon":stats.uniform(0,1)
}
clf = SVR(gamma='scale')
hyper = HyperbandSearchCV(clf, param_distributions =rf_params,cv=3,min_iter=1,max_iter=10,scoring='neg_mean_squared_error',resource_param='C')
hyper.fit(X, y)
print(hyper.best_params_)
print("MSE:"+ str(-hyper.best_score_))

{'kernel': 'poly', 'epsilon': 0.4490042156616516, 'C': 10}
MSE:70.78132735518886


In [42]:
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
rf_params = {
    'n_neighbors': range(1,20),
}
clf = KNeighborsRegressor()
hyper = HyperbandSearchCV(clf, param_distributions =rf_params,cv=3,min_iter=1,max_iter=20,scoring='neg_mean_squared_error',resource_param='n_neighbors')
hyper.fit(X, y)
print(hyper.best_params_)
print("MSE:"+ str(-hyper.best_score_))

{'n_neighbors': 6}
MSE:80.87024044795783


## HPO Algorithm 4: BO-GP

### Using skopt.BayesSearchCV

In [43]:
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
rf_params = {
    'n_estimators': Integer(10,100),
    "max_features":Integer(1,13),
    'max_depth': Integer(5,50),
    "min_samples_split":Integer(2,11),
    "min_samples_leaf":Integer(1,11),
    #"criterion":['gini','entropy']
}
clf = RandomForestRegressor(random_state=0)
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=20, scoring='neg_mean_squared_error')
Bayes.fit(X, y)
print(Bayes.best_params_)
bclf = Bayes.best_estimator_
print("MSE:"+ str(-Bayes.best_score_))

{'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'max_features': 9}
MSE:27.054063120553348


In [44]:
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
rf_params = {
    'C': Real(0,50),
    "kernel":['poly','rbf','sigmoid'],
    'epsilon': Real(0,1)
}
clf = SVR(gamma='scale')
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=20, scoring='neg_mean_squared_error')
Bayes.fit(X, y)
print(Bayes.best_params_)
print("MSE:"+ str(-Bayes.best_score_))

{'kernel': 'poly', 'epsilon': 0.16781739810509447, 'C': 43.14510166511289}
MSE:59.52440851660976


In [45]:
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
rf_params = {
    'n_neighbors': Integer(1,20),
}
clf = KNeighborsRegressor()
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=10, scoring='neg_mean_squared_error')
Bayes.fit(X, y)
print(Bayes.best_params_)
print("MSE:"+ str(-Bayes.best_score_))

{'n_neighbors': 13}
MSE:80.7723025469514


### Using skopt.gp_minimize

In [51]:
reg = RandomForestRegressor()
from skopt.space import Real, Integer
from skopt.utils import use_named_args

space  = [Integer(10, 100, name='n_estimators'),
            Integer(5, 50, name='max_depth'),
          Integer(1, 13, name='max_features'),
          Integer(2, 11, name='min_samples_split'),
          Integer(1, 11, name='min_samples_leaf'),
         #Categorical(['gini', 'entropy'], name='criterion'),
         ]

@use_named_args(space)
def objective(**params):
    reg.set_params(**params)

    return -np.mean(cross_val_score(reg, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
from skopt import gp_minimize
res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)
print("MSE:%.4f" % res_gp.fun)
print(res_gp.x)

MSE:26.2711
[18, 14, 6, 10, 4]


In [52]:
reg = SVR(gamma='scale')
from skopt.space import Real, Integer
from skopt.utils import use_named_args

space  = [Real(0, 50, name='C'),
          Categorical(['poly','rbf','sigmoid'], name='kernel'),
          Real(0, 1, name='epsilon'),
         ]

@use_named_args(space)
def objective(**params):
    reg.set_params(**params)

    return -np.mean(cross_val_score(reg, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
from skopt import gp_minimize
res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)
print("MSE:%.4f" % res_gp.fun)
print(res_gp.x)

MSE:61.2510
[37.93078121611787, 'poly', 0.47360041934665753]


In [53]:
reg = KNeighborsRegressor()
from skopt.space import Real, Integer
from skopt.utils import use_named_args

space  = [Integer(1, 20, name='n_neighbors')]

@use_named_args(space)
def objective(**params):
    reg.set_params(**params)

    return -np.mean(cross_val_score(reg, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
from skopt import gp_minimize
res_gp = gp_minimize(objective, space, n_calls=10, random_state=0)
print("MSE:%.4f" % res_gp.fun)
print(res_gp.x)

MSE:80.7412
[13]


## HPO Algorithm 5: BO-TPE

In [54]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
def objective(params):
    params = {
        'n_estimators': int(params['n_estimators']), 
        'max_depth': int(params['max_depth']),
        'max_features': int(params['max_features']),
        "min_samples_split":int(params['min_samples_split']),
        "min_samples_leaf":int(params['min_samples_leaf']),
        #"criterion":str(params['criterion'])
    }
    clf = RandomForestRegressor( **params)
    score = -np.mean(cross_val_score(clf, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
    #print("ROC-AUC {:.3f} params {}".format(score, params))

    return {'loss':score, 'status': STATUS_OK }

space = {
    'n_estimators': hp.quniform('n_estimators', 10, 100, 1),
    'max_depth': hp.quniform('max_depth', 5, 50, 1),
    "max_features":hp.quniform('max_features', 1, 13, 1),
    "min_samples_split":hp.quniform('min_samples_split',2,11,1),
    "min_samples_leaf":hp.quniform('min_samples_leaf',1,11,1),
    "criterion":hp.choice('criterion',['gini','entropy'])
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20)
print("Random Forest: Hyperopt estimated optimum {}".format(best))

100%|███████████████████████████████████████████████████| 20/20 [00:01<00:00, 13.61it/s, best loss: 26.857722814717487]
Random Forest: Hyperopt estimated optimum {'max_depth': 50.0, 'min_samples_leaf': 3.0, 'min_samples_split': 7.0, 'max_features': 5.0, 'criterion': 0, 'n_estimators': 82.0}


In [55]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
def objective(params):
    params = {
        'C': abs(float(params['C'])), 
        "kernel":str(params['kernel']),
        'epsilon': abs(float(params['epsilon'])),
    }
    clf = SVR(gamma='scale', **params)
    score = -np.mean(cross_val_score(clf, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
    #print("ROC-AUC {:.3f} params {}".format(score, params))
    print(score)
    return {'loss':score, 'status': STATUS_OK }

space = {
    'C': hp.normal('C', 0, 50),
    "kernel":hp.choice('kernel',['poly','rbf','sigmoid']),
    'epsilon': hp.normal('epsilon', 0, 1),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20)
print("Random Forest: Hyperopt estimated optimum {}".format(best))

88.96902491065731
48393.901625625724
70.84123583621884
77.22373372727031
1921.583414860842
62.76781604366425
65.95203354662758
62.127139094608374
78.11921987086193
63.75742229784455
82.73068113742441
11511.386775697205
12491.8779074914
8438.521222468902
81.28298963881998
60.803371026183214
64.85414012079134
74.65877567726007
82.64907968740728
70.32108009755741
100%|███████████████████████████████████████████████████| 20/20 [00:00<00:00, 63.23it/s, best loss: 60.803371026183214]
Random Forest: Hyperopt estimated optimum {'kernel': 0, 'epsilon': 0.3306178089802077, 'C': 23.418083354013188}


In [56]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
def objective(params):
    params = {
        'n_neighbors': abs(int(params['n_neighbors']))
    }
    clf = KNeighborsRegressor( **params)
    score = -np.mean(cross_val_score(clf, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
    #print("ROC-AUC {:.3f} params {}".format(score, params))

    return {'loss':score, 'status': STATUS_OK }

space = {
    'n_neighbors': hp.quniform('n_neighbors', 1, 20, 1),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Random Forest: Hyperopt estimated optimum {}".format(best))

100%|███████████████████████████████████████████████████| 10/10 [00:00<00:00, 147.17it/s, best loss: 80.83005201647829]
Random Forest: Hyperopt estimated optimum {'n_neighbors': 6.0}


## HPO Algorithm 6: PSO

In [61]:
import optunity
import optunity.metrics
search = {
    'n_estimators': [10, 100],
    'max_features': [1, 13],
    'max_depth': [5,50],
    "min_samples_split":[2,11],
    "min_samples_leaf":[1,11],
         }
@optunity.cross_validated(x=X, y=y, num_folds=3)
def performance(x_train, y_train, x_test, y_test,n_estimators=None, max_features=None,max_depth=None,min_samples_split=None,min_samples_leaf=None):
    # fit the model
    model = RandomForestRegressor(n_estimators=int(n_estimators),
                                   max_features=int(max_features),
                                   max_depth=int(max_depth),
                                   min_samples_split=int(min_samples_split),
                                   min_samples_leaf=int(min_samples_leaf),
                                  )
    #predictions = model.predict(x_test)
    scores=-np.mean(cross_val_score(model, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
    #return optunity.metrics.roc_auc(y_test, predictions, positive=True)
    return scores#optunity.metrics.accuracy(y_test, predictions)

optimal_configuration, info, _ = optunity.minimize(performance,
                                                  solver_name='particle swarm',
                                                  num_evals=20,
                                                   **search
                                                  )
print(optimal_configuration)
print("MSE:"+ str(info.optimum))

{'max_depth': 37.745028056718674, 'min_samples_leaf': 1.2694935979785371, 'min_samples_split': 6.3674769629093, 'n_estimators': 97.73792799268917, 'max_features': 6.383593749999999}
MSE:26.072547695737644


In [62]:
search = {
    'C': (0,50),
    'kernel':[0,3],
    'epsilon': (0, 1)
         }
@optunity.cross_validated(x=X, y=y, num_folds=3)
def performance(x_train, y_train, x_test, y_test,C=None,kernel=None,epsilon=None):
    # fit the model
    if kernel<1:
        ke='poly'
    elif kernel<2:
        ke='rbf'
    else:
        ke='sigmoid'
    model = SVR(C=float(C),
                kernel=ke,
                gamma='scale',
                epsilon=float(epsilon)
                                  )
    #predictions = model.predict(x_test)
    scores=-np.mean(cross_val_score(model, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
    #return optunity.metrics.roc_auc(y_test, predictions, positive=True)
    return scores#optunity.metrics.accuracy(y_test, predictions)

optimal_configuration, info, _ = optunity.minimize(performance,
                                                  solver_name='particle swarm',
                                                  num_evals=20,
                                                   **search
                                                  )
print(optimal_configuration)
print("MSE:"+ str(info.optimum))

{'kernel': 0.4658203125, 'epsilon': 0.4736328125, 'C': 25.341796875}
MSE:60.25879498950017


In [63]:
search = {
    'n_neighbors': [1, 20],
         }
@optunity.cross_validated(x=X, y=y, num_folds=3)
def performance(x_train, y_train, x_test, y_test,n_neighbors=None):
    # fit the model
    model = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                  )
    #predictions = model.predict(x_test)
    scores=-np.mean(cross_val_score(model, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
    #return optunity.metrics.roc_auc(y_test, predictions, positive=True)
    return scores#optunity.metrics.accuracy(y_test, predictions)

optimal_configuration, info, _ = optunity.minimize(performance,
                                                  solver_name='particle swarm',
                                                  num_evals=10,
                                                   **search
                                                  )
print(optimal_configuration)
print("MSE:"+ str(info.optimum))

{'n_neighbors': 13.84912109375}
MSE:80.74121499347262


## HPO Algorithm 7: Genetic Algorithm

### Using DEAP

In [64]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange

rf_params = {
    'n_estimators': range(10,100),
    "max_features":range(1,13),
    'max_depth': range(5,50),
    "min_samples_split":range(2,11),
    "min_samples_leaf":range(1,11),
    #Categorical(name='criterion', categories=['gini','entropy'])#
    #"criterion":['gini','entropy']
}
clf = RandomForestRegressor(random_state=0)
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="neg_mean_squared_error",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("MSE:"+ str(ga1.best_score_))

Types [1, 1, 1, 1, 1] and maxint [44, 9, 8, 11, 89] detected
--- Evolve in 4374000 possible combinations ---
gen	nevals	avg     	min    	max     	std    
0  	10    	-29.5802	-44.533	-25.5803	5.11814
1  	5     	-26.9872	-28.5839	-25.5803	1.14967
2  	5     	-26.0642	-26.5163	-25.5803	0.270993
3  	3     	-25.9576	-27.1058	-25.5803	0.465954
4  	7     	-25.7154	-26.5163	-25.5803	0.294316
5  	7     	-25.5803	-25.5803	-25.5803	3.55271e-15
Best individual is: {'max_depth': 29, 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 12, 'max_features': 8}
with fitness: -25.58028227797807
{'max_depth': 29, 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 12, 'max_features': 8}
MSE:-25.58028227797807


In [65]:
rf_params = {
    'C': np.random.uniform(0,50,1000),
    "kernel":['poly','rbf','sigmoid'],
    'epsilon': np.random.uniform(0,1,100),
}
clf = SVR(gamma='scale')
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="neg_mean_squared_error",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("MSE:"+ str(ga1.best_score_))

Types [1, 2, 2] and maxint [2, 99, 999] detected
--- Evolve in 300000 possible combinations ---
gen	nevals	avg     	min     	max     	std    
0  	10    	-791.408	-4580.37	-60.3024	1407.48
1  	7     	-437.988	-3673.51	-60.3024	1078.66
2  	6     	-70.1584	-80.5238	-60.3024	6.80748
3  	7     	-67.2654	-79.4701	-60.3024	7.20768
4  	5     	-61.7897	-69.2764	-60.3024	2.75386
5  	8     	-60.3024	-60.3024	-60.3024	0      
Best individual is: {'kernel': 'poly', 'epsilon': 0.34123803949334774, 'C': 25.638198236692972}
with fitness: -60.30244732642168
{'kernel': 'poly', 'epsilon': 0.34123803949334774, 'C': 25.638198236692972}
MSE:-60.30244732642168


In [66]:
rf_params = {
    'n_neighbors': range(1,20),
}
clf = KNeighborsRegressor()
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="neg_mean_squared_error",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("MSE:"+ str(ga1.best_score_))

Types [1] and maxint [18] detected
--- Evolve in 19 possible combinations ---
gen	nevals	avg     	min     	max     	std     
0  	10    	-82.4511	-83.3224	-80.8702	0.775591
1  	3     	-81.8812	-82.9839	-80.8702	0.718567
2  	6     	-81.1339	-81.5293	-80.8702	0.322889
3  	9     	-81.1776	-83.2849	-80.8702	0.729407
4  	7     	-80.8702	-80.8702	-80.8702	0       
5  	7     	-80.9662	-81.8295	-80.8702	0.287764
Best individual is: {'n_neighbors': 6}
with fitness: -80.87024044795783
{'n_neighbors': 6}
MSE:-80.87024044795783


### Using TPOT

In [69]:
from tpot import TPOTRegressor

parameters = {
    'n_estimators': range(20,200),
    "max_features":range(1,13),
    'max_depth': range(10,100),
    "min_samples_split":range(2,11),
    "min_samples_leaf":range(1,11),
    #"criterion":['gini','entropy']
             }
               
ga2 = TPOTRegressor(generations= 3, population_size= 10, offspring_size= 5,
                                 verbosity= 3, early_stop= 5,
                                 config_dict=
                                 {'sklearn.ensemble.RandomForestRegressor': parameters}, 
                                 cv = 3, scoring = 'neg_mean_squared_error')
ga2.fit(X, y)

1 operators have been imported by TPOT.
Generation 1 - Current Pareto front scores:
-1	-27.250793005044518	RandomForestRegressor(input_matrix, RandomForestRegressor__max_depth=78, RandomForestRegressor__max_features=5, RandomForestRegressor__min_samples_leaf=2, RandomForestRegressor__min_samples_split=9, RandomForestRegressor__n_estimators=168)
-2	-25.332016721242965	RandomForestRegressor(RandomForestRegressor(input_matrix, RandomForestRegressor__max_depth=62, RandomForestRegressor__max_features=4, RandomForestRegressor__min_samples_leaf=5, RandomForestRegressor__min_samples_split=6, RandomForestRegressor__n_estimators=34), RandomForestRegressor__max_depth=78, RandomForestRegressor__max_features=5, RandomForestRegressor__min_samples_leaf=2, RandomForestRegressor__min_samples_split=9, RandomForestRegressor__n_estimators=168)

Generation 2 - Current Pareto front scores:
-1	-27.250793005044518	RandomForestRegressor(input_matrix, RandomForestRegressor__max_depth=78, RandomForestRegressor__

TPOTRegressor(config_dict={'sklearn.ensemble.RandomForestRegressor': {'max_depth': range(10, 100), 'min_samples_leaf': range(1, 11), 'min_samples_split': range(2, 11), 'max_features': range(1, 13), 'n_estimators': range(20, 200)}},
       crossover_rate=0.1, cv=3, disable_update_check=False, early_stop=5,
       generations=3, max_eval_time_mins=5, max_time_mins=None,
       memory=None, mutation_rate=0.9, n_jobs=1, offspring_size=5,
       periodic_checkpoint_folder=None, population_size=10,
       random_state=None, scoring='neg_mean_squared_error', subsample=1.0,
       template=None, use_dask=False, verbosity=3, warm_start=False)

In [70]:
t1=time.time()
from tpot import TPOTRegressor

parameters = {
    'C': np.random.uniform(0,50,1000),
    "kernel":['poly','rbf','sigmoid'],
    'epsilon': np.random.uniform(0,1,100),
    'gamma': ['scale']
             }
               
ga2 = TPOTRegressor(generations= 3, population_size= 10, offspring_size= 5,
                                 verbosity= 3, early_stop= 5,
                                 config_dict=
                                 {'sklearn.svm.SVR': parameters}, 
                                 cv = 3, scoring = 'neg_mean_squared_error')
ga2.fit(X, y)

1 operators have been imported by TPOT.
Generation 1 - Current Pareto front scores:
-1	-76.22342756605103	SVR(CombineDFs(input_matrix, input_matrix), SVR__C=9.66289149097888, SVR__epsilon=0.6491076078954555, SVR__gamma=scale, SVR__kernel=rbf)
-2	-61.22653072468754	SVR(SVR(CombineDFs(input_matrix, input_matrix), SVR__C=2.988848947755768, SVR__epsilon=0.31470079088508107, SVR__gamma=scale, SVR__kernel=poly), SVR__C=49.38466344757709, SVR__epsilon=0.9554593389709706, SVR__gamma=scale, SVR__kernel=poly)

Generation 2 - Current Pareto front scores:
-1	-73.46312983258913	SVR(input_matrix, SVR__C=9.66289149097888, SVR__epsilon=0.6491076078954555, SVR__gamma=scale, SVR__kernel=poly)
-2	-61.22653072468754	SVR(SVR(CombineDFs(input_matrix, input_matrix), SVR__C=2.988848947755768, SVR__epsilon=0.31470079088508107, SVR__gamma=scale, SVR__kernel=poly), SVR__C=49.38466344757709, SVR__epsilon=0.9554593389709706, SVR__gamma=scale, SVR__kernel=poly)

Generation 3 - Current Pareto front scores:
-1	-59.46

TPOTRegressor(config_dict={'sklearn.svm.SVR': {'kernel': ['poly', 'rbf', 'sigmoid'], 'gamma': ['scale'], 'epsilon': array([0.25137, 0.0217 , ..., 0.7522 , 0.4685 ]), 'C': array([40.05907, 22.18248, ..., 33.62995,  2.73409])}},
       crossover_rate=0.1, cv=3, disable_update_check=False, early_stop=5,
       generations=3, max_eval_time_mins=5, max_time_mins=None,
       memory=None, mutation_rate=0.9, n_jobs=1, offspring_size=5,
       periodic_checkpoint_folder=None, population_size=10,
       random_state=None, scoring='neg_mean_squared_error', subsample=1.0,
       template=None, use_dask=False, verbosity=3, warm_start=False)

In [71]:
t1=time.time()
from tpot import TPOTRegressor

parameters = {
    'n_neighbors': range(1,20),
             }
               
ga2 = TPOTRegressor(generations= 3, population_size= 10, offspring_size= 5,
                                 verbosity= 3, early_stop= 5,
                                 config_dict=
                                 {'sklearn.neighbors.KNeighborsRegressor': parameters}, 
                                 cv = 3, scoring = 'neg_mean_squared_error')
ga2.fit(X, y)

1 operators have been imported by TPOT.
Generation 1 - Current Pareto front scores:
-1	-80.83005201647829	KNeighborsRegressor(input_matrix, KNeighborsRegressor__n_neighbors=6)

Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
Generation 2 - Current Pareto front scores:
-1	-80.83005201647829	KNeighborsRegressor(input_matrix, KNeighborsRegressor__n_neighbors=6)

Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
Generation 3 - Current Pareto front scores:
-1	-80.83005201647829	KNeighborsRegressor(input_matrix, KNeighborsRegressor__n_neighbors=6)


The optimized pipeline was not improved after evaluating 5 more generations. Will end the optimization process.

TPOT closed prematurely. Will use the current best pipeline.


TPOTRegressor(config_dict={'sklearn.neighbors.KNeighborsRegressor': {'n_neighbors': range(1, 20)}},
       crossover_rate=0.1, cv=3, disable_update_check=False, early_stop=5,
       generations=3, max_eval_time_mins=5, max_time_mins=None,
       memory=None, mutation_rate=0.9, n_jobs=1, offspring_size=5,
       periodic_checkpoint_folder=None, population_size=10,
       random_state=None, scoring='neg_mean_squared_error', subsample=1.0,
       template=None, use_dask=False, verbosity=3, warm_start=False)